In [1]:
import pandas as pd
from DataLoader_ey import data_loader 

In [2]:
df = data_loader()
df.head()

,balance_,duration_,history_,purpose_,amount_,savings_,employment_,available_income_,other_debtors_,residence_,assets_,age_,other_loans_,housing_,previous_loans_,job_,people_liable_,telephone_,label
0,no account,6,paid back previous loans at this bank,furniture,1169.0,above 1000 EUR,more than 7 years,less than 20%,none,more than 7 years,real estate,67,no additional loans,own,2 or 3,skilled,0 to 2,yes,Approved
1,no balance,48,paid back all previous loans,furniture,5951.0,no savings account at this bank,between 1 and 4 years,between 25 and 35%,none,between 1 and 4 years,real estate,22,no additional loans,own,1,skilled,0 to 2,none,Rejected
2,above 200 EUR,12,paid back previous loans at this bank,repair,2096.0,no savings account at this bank,between 4 and 7 years,between 25 and 35%,none,between 4 and 7 years,real estate,49,no additional loans,own,1,unskilled (permanent resident),3 and more,none,Approved
3,no account,42,paid back all previous loans,used car,7882.0,no savings account at this bank,between 4 and 7 years,between 25 and 35%,guarantor,more than 7 years,life insurance,45,no additional loans,for free,1,skilled,3 and more,none,Approved
4,no account,24,no problems with current loans,other,4870.0,no savings account at this bank,between 1 and 4 years,between 20 and 25%,none,more than 7 years,none,53,no additional loans,for free,2 or 3,skilled,3 and more,none,Rejected


# Requirements to put data in a database

## Phase 1
For the database, in addition to the `label` column should be replaced by a `NN_recommendation` column and will use the models prediction for each application; the output of the neural network (`smote_ey.tf`) should also be used for a `NN_confidence` column, which is a float between 0 and 1 for each row. A confidence of 1 would imply that the recommendation is made with a certainty of 100% (which implies the independence of the prediction value approve or reject)
- the output of the model is one neuron, which means that the confidence could be interpreted in different ways.
- first, find out which number (0 or 1) corresponds to approved and rejected in the model ouput.
- Check literature to see what is the best way to interpret a one-dimensional binary classification output in terms of confidence for neural networks, evaluate between the following possibilities:
    - <b>possibility 1</b>: if the output of the neuron is $p$ ($\in [0, 1]$), then the confidence should be $p$ (or $1-p$, depending on the mapping to approve and reject)
    - <b>possibility 2</b>: if the output of the neuron is $p$ ($\in [0, 1]$), then the confidence should correspond to $(p-0.5)\cdot2$ (or $(0.5-p) \cdot 2$, depending on the mapping to approve and reject) which would lead to a confidence of 0% if the model's output is 0.5
    - Maybe a better way?

Task:
- Modify `API/DataLoader_ey.py` in by adding a new function which takes the csv file `Data/german.csv` as input and returns it as a pandas DataFrame with the recommendations instead of the labels and the confidence values (Maybe add an option to also keep the labels, which might be useful later). 

<b>Goal:</b> the dataFrame is filled with the recommendation strings "approve" or "reject" and with the confidence values. The DataFrame could now be sent to a sqlite database with the pandas `DataFrame.to_sql()` function

## Phase 2       
For the actual database use SQLite (python module: `sqlite3`, https://docs.python.org/3/library/sqlite3.html).
Create the database. To achieve this, write a file named `database_creation.py` which could always be used to reset the database to the initial 1000 instances. There should be a function which saves a database file with the dataFrame from phase 1 to a separate folder & a function which would make it possible to replace the (tables of, evaluate which is smarter for later use)running database with the one created in the seperate folder (resetting).



In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   balance_           1000 non-null   object 
 1   duration_          1000 non-null   int64  
 2   history_           1000 non-null   object 
 3   purpose_           1000 non-null   object 
 4   amount_            1000 non-null   float64
 5   savings_           1000 non-null   object 
 6   employment_        1000 non-null   object 
 7   available_income_  1000 non-null   object 
 8   other_debtors_     1000 non-null   object 
 9   residence_         1000 non-null   object 
 10  assets_            1000 non-null   object 
 11  age_               1000 non-null   int64  
 12  other_loans_       1000 non-null   object 
 13  housing_           1000 non-null   object 
 14  previous_loans_    1000 non-null   object 
 15  job_               1000 non-null   object 
 16  people_liable_     1000 n

Dataset correctly has 1000 non-null entries